# KBO
### Selenium을 사용하여 데이터 수집하는 방법을 살펴보겠습니다.


In [1]:
!pip install webdriver-manager

In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

* 정규식 패턴을 사용합니다. 
    * 아래 정규식 패턴은 playerId의 숫자값을 추출하는 정규식입니다.

In [4]:
pattern = re.compile("playerId=([0-9]+)")

* Selenium을 사용하기 위해서는 브라우저의 드라이버를 설정해야 합니다. 
    * 이번 예제는 크롬 브라우저를 사용하기 때문에 크롬 드라이버를 선택해줍니다. 
    * 다운로드
    * https://chromedriver.chromium.org/downloads

In [8]:
s = Service("C:/chrome/chromedriver.exe")

driver = webdriver.Chrome(service=s)

In [9]:
driver.get("http://www.koreabaseball.com")

In [10]:
driver.find_element(by=By.CSS_SELECTOR, 
                    value="#popupVideo > img").click()

In [6]:
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(8)
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(10)

In [11]:
driver.find_element(by=By.CSS_SELECTOR, 
                     value="#lnb > li:nth-child(4) > a").click()

In [12]:
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"

In [13]:
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"

In [14]:
import time

In [15]:
playid = []
for x in range(2,12):
    for_1 = select_team.format(x)
    driver.find_element(by=By.CSS_SELECTOR, value=for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(by=By.CSS_SELECTOR, value=f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print (e)
        time.sleep(2)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#cphContents_cphContents_cphContents_ucPager_btnNo5"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x00568BD3+2395091]
	Ordinal0 [0x004FACA1+1944737]
	Ordinal0 [0x003ED008+839688]
	Ordinal0 [0x00419BFD+1022973]
	Ordinal0 [0x00419EBB+1023675]
	Ordinal0 [0x00446A92+1206930]
	Ordinal0 [0x00434C04+1133572]
	Ordinal0 [0x00444D62+1199458]
	Ordinal0 [0x004349D6+1133014]
	Ordinal0 [0x0040EF76+978806]
	Ordinal0 [0x0040FE86+982662]
	GetHandleVerifier [0x0071C912+1719138]
	GetHandleVerifier [0x007CB2CD+2434333]
	GetHandleVerifier [0x00604001+569937]
	GetHandleVerifier [0x00603066+565942]
	Ordinal0 [0x0050265B+1975899]
	Ordinal0 [0x005072A8+1995432]
	Ordinal0 [0x00507395+1995669]
	Ordinal0 [0x005102F1+2032369]
	BaseThreadInitThunk [0x76CAFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77E97A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77E97A4E+238]

Message: no such element: Unable 

In [16]:
len(playid)

829

In [17]:
import pickle

In [18]:
with open("./kbo.pkl", "wb") as f:
    pickle.dump(playid,f)

### 선수들의 세부 항목들을 수집합니다.

In [19]:
r = requests.get("https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=62356")

* 이미지를 다운로드 받고 싶다면 아래 코드를 사용합니다.

In [20]:
request.urlretrieve("http:"+BS(r.text).find("div",\
            class_ = "player_basic").find("img")['src'], "./김규민.jpg")

('./김규민.jpg', <http.client.HTTPMessage at 0x19b6e680dc0>)

### 사용자 함수를 작성하여 선수들의 세부 정보를 수집합니다.

In [23]:
def playerID(id_):
    play_dict = {}
    url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}".format(id_)
    r = requests.get(url)
    #print (r.url)
    for x in BS(r.text).find("div",\
                class_ = "player_basic").findAll("li"):
        rt = [y.strip() for y in x.text.strip().split(":")]
        play_dict[rt[0]] = rt[1]
    play_dict['Team'] = BS(r.text).find("h4", id="h4Team").text
    return play_dict

In [22]:
playid[0]

'69158'

In [24]:
playerID(playid[0])

{'선수명': '강정현',
 '등번호': 'No.100',
 '생년월일': '1995년 09월 23일',
 '포지션': '투수(우투우타)',
 '신장/체중': '180cm/90kg',
 '경력': '남부민초-대신중-부경고-원광대-LG-상무',
 '입단 계약금': '7000만원',
 '연봉': '3300만원',
 '지명순위': '19 LG 2차 4라운드 35순위',
 '입단년도': '19LG',
 'Team': 'LG 트윈스'}